STEP 1: Install Required Libraries (Colab Cell 1)

In [13]:
!pip install flask pyngrok pillow


STEP 2: Create Project Structure (Colab Cell 2)

In [14]:
import os

base_dir = "/content/lost_found_app"
templates_dir = os.path.join(base_dir, "templates")
static_dir = os.path.join(base_dir, "static")
upload_dir = os.path.join(static_dir, "uploads")

os.makedirs(templates_dir, exist_ok=True)
os.makedirs(upload_dir, exist_ok=True)

print("Project folders created")


Project folders created


🔹 STEP 3: Flask Backend (app.py) (Colab Cell 3)

In [15]:
%%writefile /content/lost_found_app/app.py
from flask import Flask, render_template, request, redirect, url_for
import sqlite3, os
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'

DB = "lostfound.db"

def init_db():
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    c.execute("""
    CREATE TABLE IF NOT EXISTS items (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        description TEXT,
        category TEXT,
        status TEXT,
        image TEXT
    )
    """)
    conn.commit()
    conn.close()

init_db()

@app.route('/')
def index():
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    c.execute("SELECT * FROM items")
    items = c.fetchall()
    conn.close()
    return render_template("index.html", items=items)

@app.route('/add', methods=['GET', 'POST'])
def add_item():
    if request.method == 'POST':
        name = request.form['name']
        description = request.form['description']
        category = request.form['category']
        status = request.form['status']
        image = request.files['image']

        filename = secure_filename(image.filename)
        image.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

        conn = sqlite3.connect(DB)
        c = conn.cursor()
        c.execute("INSERT INTO items (name, description, category, status, image) VALUES (?, ?, ?, ?, ?)",
                  (name, description, category, status, filename))
        conn.commit()
        conn.close()

        return redirect(url_for('index'))

    return render_template("add.html")

if __name__ == "__main__":
    app.run()


Overwriting /content/lost_found_app/app.py


STEP 4: Frontend – Home Page (index.html) (Colab Cell 4)

In [16]:
%%writefile /content/lost_found_app/templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>Lost & Found Portal</title>
    <style>
        body { font-family: Arial; background:#f4f4f4; }
        .card { background:white; padding:15px; margin:15px; border-radius:10px; width:250px; display:inline-block; }
        img { width:100%; height:180px; object-fit:cover; }
        a { text-decoration:none; color:white; background:green; padding:10px; border-radius:5px; }
    </style>
</head>
<body>

<h1 align="center">Lost & Found Portal</h1>
<center><a href="/add">➕ Add Lost/Found Item</a></center>

<div style="text-align:center">
{% for item in items %}
<div class="card">
    <img src="{{ url_for('static', filename='uploads/' + item[5]) }}">
    <h3>{{ item[1] }}</h3>
    <p>{{ item[2] }}</p>
    <b>{{ item[3] }} | {{ item[4] }}</b>
</div>
{% endfor %}
</div>

</body>
</html>


Overwriting /content/lost_found_app/templates/index.html


STEP 5: Frontend – Add Item Page (add.html) (Colab Cell 5)

In [17]:
%%writefile /content/lost_found_app/templates/add.html
<!DOCTYPE html>
<html>
<head>
    <title>Add Item</title>
</head>
<body style="font-family:Arial">

<h2>Add Lost / Found Item</h2>

<form method="POST" enctype="multipart/form-data">
    Item Name:<br>
    <input type="text" name="name" required><br><br>

    Description:<br>
    <textarea name="description" required></textarea><br><br>

    Category:<br>
    <select name="category">
        <option>Mobile</option>
        <option>Wallet</option>
        <option>Documents</option>
        <option>Other</option>
    </select><br><br>

    Status:<br>
    <select name="status">
        <option>Lost</option>
        <option>Found</option>
    </select><br><br>

    Image:<br>
    <input type="file" name="image" required><br><br>

    <button type="submit">Submit</button>
</form>

</body>
</html>


Overwriting /content/lost_found_app/templates/add.html


STEP 6: Run App with Public Link (ngrok) (Colab Cell 6)

In [ ]:
from pyngrok import ngrok
import os
from google.colab import userdata

os.chdir("/content/lost_found_app")

# Get ngrok authtoken securely from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
else:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")

public_url = ngrok.connect(5000)
print("Public URL:", public_url)

!python app.py

Public URL: NgrokTunnel: "https://unstocked-hearteningly-aspen.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jan/2026 14:36:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2026 14:36:03] "GET /add HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2026 15:06:37] "POST /add HTTP/1.1" 302 -
127.0.0.1 - - [06/Jan/2026 15:06:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2026 15:06:39] "GET /static/uploads/DEFECTED_PIPILINE.jpg HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2026 15:16:47] "GET /add HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2026 15:23:58] "POST /add HTTP/1.1" 302 -
127.0.0.1 - - [06/Jan/2026 15:23:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2026 15:23:59] "GET /static/uploads/DEFECTED_PIPILINE.jpg HTTP/1.1" 304 -
127.0.0.1 - - [06/Jan/2026 15:23:59] "GET /static/uploads/gas_pipeline.png HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2026 15:24:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - 